# 1. Load the Data

In [2]:
import pandas as pd

# Load the filtered dataset with tab separator
data = pd.read_csv('cereal_spain.csv', sep='\t')

# Display the first few rows of the dataset
print(data.head())

            code  producer_product_id producer_version_id  lc product_name_ar  \
0  3168930010265                  NaN                 NaN  fr             NaN   
1  3229820019307                  NaN                 NaN  fr             NaN   
2       20003166                  NaN                 NaN  en             NaN   
3  5411188124689                  NaN                 NaN  de             NaN   
4  3229820782560                  NaN                 NaN  fr             NaN   

  product_name_at product_name_bg        product_name_ca product_name_cs  \
0             NaN             NaN                    NaN             NaN   
1             NaN             NaN                    NaN             NaN   
2             NaN             NaN  Flocs de civada suaus             NaN   
3             NaN             NaN                    NaN             NaN   
4             NaN             NaN                    NaN             NaN   

  product_name_da  ... sources_fields:org-database-usda:

C:\Users\arsen\AppData\Local\Temp\ipykernel_860\3728654597.py:4: DtypeWarning: Columns (0,2,4,5,6,8,9,11,14,15,16,17,18,19,21,22,23,24,25,26,27,28,30,31,32,33,34,35,36,37,38,39,40,41,42,43,45,46,47,48,51,52,53,54,55,57,58,59,60,61,62,63,64,65,66,67,68,75,85,88,89,91,92,93,96,97,98,99,100,101,102,105,106,107,109,110,112,115,116,117,118,119,120,121,122,123,124,125,126,127,129,130,131,132,133,134,135,136,141,147,151,155,159,163,167,171,175,179,183,189,191,193,195,197,199,201,205,211,213,215,217,219,221,227,229,231,233,235,237,239,241,243,245,249,251,255,259,263,267,271,273,275,277,279,281,285,287,291,295,299,303,305,309,311,313,315,317,321,323,325,327,336,337,338,342,343,344,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,378,379,380,381,382,383,384,385,386,387,388,389,390,407,408,409,411,412,415,416,417,418,419,420) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('cereal_spain.c

# 2. Explore the dataset

Check the column names to identify which we can drop

In [3]:
# Prevent output from being truncated by rows or columns
pd.set_option('display.max_rows', None)

# Print the list of the columns in the dataset
print(data.columns.to_list())

['code', 'producer_product_id', 'producer_version_id', 'lc', 'product_name_ar', 'product_name_at', 'product_name_bg', 'product_name_ca', 'product_name_cs', 'product_name_da', 'product_name_de', 'product_name_el', 'product_name_en', 'product_name_es', 'product_name_et', 'product_name_fa', 'product_name_fi', 'product_name_fr', 'product_name_hr', 'product_name_hu', 'product_name_it', 'product_name_ko', 'product_name_la', 'product_name_lt', 'product_name_lv', 'product_name_nb', 'product_name_nl', 'product_name_no', 'product_name_pl', 'product_name_pt', 'product_name_ro', 'product_name_ru', 'product_name_sk', 'product_name_sl', 'product_name_sr', 'product_name_sv', 'product_name_tr', 'product_name_uk', 'product_name_xx', 'product_name_zh', 'abbreviated_product_name_fr', 'generic_name_ar', 'generic_name_bg', 'generic_name_bs', 'generic_name_ca', 'generic_name_cs', 'generic_name_da', 'generic_name_de', 'generic_name_el', 'generic_name_en', 'generic_name_es', 'generic_name_et', 'generic_name_f

Verify that "code" is a primary key

In [4]:
# Check if the number of the rows in the dataset is the same as the number of the unique values in the 'code' column
print(data.shape[0] == data['code'].nunique())

False


Find out what the duplicated values are

In [5]:
# Print the duplicated values in the 'code' column
duplicated_codes = data[data.duplicated(subset='code', keep=False)]
print(duplicated_codes)


               code  producer_product_id producer_version_id  lc  \
5084  5906827017304                  NaN                 NaN  es   
5103  5906827017304                  NaN                 NaN  es   

     product_name_ar product_name_at product_name_bg product_name_ca  \
5084             NaN             NaN             NaN             NaN   
5103             NaN             NaN             NaN             NaN   

     product_name_cs product_name_da  ...  \
5084             NaN             NaN  ...   
5103             NaN             NaN  ...   

     sources_fields:org-database-usda:publication_date  \
5084                                               NaN   
5103                                               NaN   

     sources_fields:org-gs1:gln sources_fields:org-gs1:gpcCategoryCode  \
5084                        NaN                                    NaN   
5103                        NaN                                    NaN   

     sources_fields:org-gs1:gpcCategoryName 

Compare the values for this duplicated code in the other columns

In [6]:
# Check if the values in all columns of these rows are the same
duplicated_grouped = duplicated_codes.groupby('code').nunique()

# Print the columns with differences in the duplicated rows
different_columns = duplicated_grouped.loc[:, (duplicated_grouped > 1).any(axis=0)]
print(different_columns)

               product_name_es  quantity  serving_size  packaging  \
code                                                                
5906827017304                2         2             2          2   

               packaging_tags  categories  categories_tags  \
code                                                         
5906827017304               2           2                2   

               ingredients_text_es  energy_value  energy_unit  \
code                                                            
5906827017304                    2             2            2   

               packaging_1_material  off:ecoscore_grade  \
code                                                      
5906827017304                     2                   2   

               off:ecoscore_data.adjustments.packaging.value  \
code                                                           
5906827017304                                              2   

               off:ecoscore_data.adjus

Explore the columns in more detail

In [7]:
# Print rows with differences for detailed inspection
for code in duplicated_codes['code'].unique():
    duplicated_rows = duplicated_codes[duplicated_codes['code'] == code]
    if not duplicated_rows.iloc[0].equals(duplicated_rows.iloc[1]):
        print(f"Differences for code {code}:")
        # Iterate over columns and print only the columns where values are different
        for column in different_columns.columns:
            value_row1 = duplicated_rows.iloc[0][column]
            value_row2 = duplicated_rows.iloc[1][column]
            # Check for NaN values
            if pd.isna(value_row1) and pd.isna(value_row2):
                continue  # Skip if both are NaN
            if value_row1 != value_row2:
                print(f"    Column: {column}")
                print(f"        Row 1: {value_row1}")
                print(f"        Row 2: {value_row2}")


Differences for code 5906827017304:
    Column: product_name_es
        Row 1: Trendy Lunch
        Row 2: Trendy Lunch lentejas verdes con bulgur y cebada
    Column: quantity
        Row 1: 400g
        Row 2: 400 g
    Column: serving_size
        Row 1: 100g
        Row 2: 100 g
    Column: packaging
        Row 1: Plástico, en:container
        Row 2: Caja, en:Cardboard
    Column: packaging_tags
        Row 1: en:plastic,en:container
        Row 2: en:box,en:cardboard
    Column: categories
        Row 1: Alimentos y bebidas de origen vegetal, Alimentos de origen vegetal, Leguminosas y derivados, Cereales y patatas, Leguminosas, Semillas, Cereales y derivados, Granos de leguminosas, Cereales en grano, Legumbres secas
        Row 2: Alimentos y bebidas de origen vegetal, Alimentos de origen vegetal, Leguminosas y derivados, Cereales y patatas, Leguminosas, Semillas, Cereales y derivados, Granos de leguminosas, Legumbres secas, Grañones, Lentejas, Lentejas verdes, Grañones de trigo

In the provided output, Row 2 seems to have more detailed information in several columns (e.g., 'product_name_es', 'categories', 'ingredients_text_es'). So we will keep this row and delete the other one.

In [8]:
# Filter out duplicate rows based on the 'code' column, keeping the second occurrence
data_no_duplicates = data.drop_duplicates(subset='code', keep='last')

# Check if the number of the rows in the new dataset is the same as the number of the unique values in the 'code' column
print(data_no_duplicates.shape[0] == data_no_duplicates['code'].nunique())

True


Delete the columns that are not interesting for the analysis.

I decided to drop columns related to product identifiers, language-specific names, ingredient texts, unused or less relevant information such as conservation conditions, warnings, preparations, recipes, customer service, and links. These columns were deemed less essential for the main analysis goals of nutritional analysis, regional analysis, and comparing food categories.

Having fewer columns should help analyze the structure of the dataframe better.

In [9]:
# List of columns to drop
columns_to_drop = [
    'producer_product_id', 'producer_version_id', 'emb_codes', 'emb_codes_tags',
    'product_name_ar', 'product_name_at', 'product_name_bg', 'product_name_ca', 'product_name_cs', 
    'product_name_da', 'product_name_de', 'product_name_el', 'product_name_et', 'product_name_fa', 
    'product_name_fi', 'product_name_fr', 'product_name_hr', 'product_name_hu', 'product_name_it', 
    'product_name_ko', 'product_name_la', 'product_name_lt', 'product_name_lv', 'product_name_nb', 
    'product_name_nl', 'product_name_no', 'product_name_pl', 'product_name_pt', 'product_name_ro', 
    'product_name_ru', 'product_name_sk', 'product_name_sl', 'product_name_sr', 'product_name_sv', 
    'product_name_tr', 'product_name_uk', 'product_name_xx', 'product_name_zh', 'abbreviated_product_name_fr', 
    'generic_name_ar', 'generic_name_bg', 'generic_name_bs', 'generic_name_ca', 'generic_name_cs', 
    'generic_name_da', 'generic_name_de', 'generic_name_el', 'generic_name_et', 'generic_name_fi', 
    'generic_name_fr', 'generic_name_hr', 'generic_name_hu', 'generic_name_it', 'generic_name_lt', 
    'generic_name_lv', 'generic_name_nl', 'generic_name_no', 'generic_name_pl', 'generic_name_pt', 
    'generic_name_ro', 'generic_name_ru', 'generic_name_sk', 'generic_name_sl', 'generic_name_sr', 
    'generic_name_sv', 'ingredients_text_ar', 'ingredients_text_bg', 'ingredients_text_bs', 'ingredients_text_ca', 
    'ingredients_text_cs', 'ingredients_text_da', 'ingredients_text_de', 'ingredients_text_el', 'ingredients_text_et', 
    'ingredients_text_fi', 'ingredients_text_fr', 'ingredients_text_gl', 'ingredients_text_hr', 'ingredients_text_hu', 
    'ingredients_text_it', 'ingredients_text_la', 'ingredients_text_lt', 'ingredients_text_lv', 'ingredients_text_nl', 
    'ingredients_text_no', 'ingredients_text_pl', 'ingredients_text_pt', 'ingredients_text_ro', 'ingredients_text_ru', 
    'ingredients_text_sk', 'ingredients_text_sl', 'ingredients_text_sr', 'ingredients_text_sv', 'ingredients_text_uk', 
    'ingredients_text_zh',
    'lc', 'obsolete', 'obsolete_since_date', 'origin_de', 'origin_en', 'origin_es', 'origin_fr', 'origin_it', 
    'origin_nl', 'conservation_conditions_de', 'conservation_conditions_en', 'conservation_conditions_es', 
    'conservation_conditions_fr', 'conservation_conditions_it', 'conservation_conditions_nl', 
    'conservation_conditions_pl', 'conservation_conditions_ro', 'warning_de', 'warning_en', 'warning_es', 
    'warning_fr', 'warning_it', 'warning_nl', 'warning_pl', 'preparation_de', 'preparation_en', 'preparation_es', 
    'preparation_fr', 'preparation_it', 'preparation_nl', 'preparation_pl', 'preparation_ro', 'recipe_idea_es', 
    'recipe_idea_fr', 'recipe_idea_it', 'recipe_idea_nl', 'recipe_idea_pl', 'recipe_idea_ro', 'customer_service_en', 
    'customer_service_es', 'customer_service_fr', 'customer_service_it', 'customer_service_nl', 'customer_service_pl', 
    'customer_service_ro', 'link'
]

# Drop specified columns from data_no_duplicates
data_filtered_columns = data_no_duplicates.drop(columns=columns_to_drop)

Let's also drop the columns with more than 70% NaN values, which is a common threshold.

In [10]:
# Calculate the percentage of NaN values in each column
nan_percentages = (data_filtered_columns.isnull().sum() / len(data_filtered_columns)) * 100

# Filter columns where NaN percentage is greater than 70%
columns_to_drop = nan_percentages[nan_percentages > 70].index

# Drop the columns from the DataFrame
data_filtered_columns = data_filtered_columns.drop(columns=columns_to_drop, axis=1)

Explore the distribution of the values in the remaining columns

In [11]:
# Get basic statistics
print(data_filtered_columns.describe())

# Get information about the dataset
print(data_filtered_columns.info())

       energy-kcal_value    fat_value  saturated-fat_value  \
count        6076.000000  6224.000000          6199.000000   
mean          339.218233     7.650505             1.876913   
std           132.708114    12.681262             3.192181   
min             0.000000     0.000000             0.000000   
25%           341.000000     1.500000             0.300000   
50%           357.000000     2.600000             0.500000   
75%           388.000000     7.600000             1.800000   
max          1458.000000   100.000000            62.290000   

       carbohydrates_value  sugars_value  fiber_value  proteins_value  \
count          6221.000000   6188.000000  3116.000000     6224.000000   
mean             55.752087      7.013855     5.574097        9.400642   
std              25.263235      9.631113     5.719943        5.779414   
min               0.000000      0.000000     0.000000        0.000000   
25%              40.700001      0.900000     2.100000        6.500000   
50%

In [12]:
# Unique values in this column off:ecoscore_data.missing_key_data
print(data_filtered_columns['off:ecoscore_data.missing_key_data'].unique())

[nan  1.]


Interesting insight: after these adjustments the key ecoscore data is missing for all products.

In [13]:
# Give me the list of columns in the dataset
print(data_filtered_columns.columns.to_list())

['code', 'product_name_es', 'quantity', 'brands', 'brands_tags', 'categories', 'categories_tags', 'labels', 'labels_tags', 'countries', 'countries_tags', 'stores', 'stores_tags', 'ingredients_text_es', 'allergens', 'allergens_tags', 'nutrition_data_per', 'nutrition_data_prepared_per', 'energy-kcal_value', 'energy-kcal_unit', 'fat_value', 'fat_unit', 'saturated-fat_value', 'saturated-fat_unit', 'carbohydrates_value', 'carbohydrates_unit', 'sugars_value', 'sugars_unit', 'fiber_value', 'fiber_unit', 'proteins_value', 'proteins_unit', 'salt_value', 'salt_unit', 'sodium_value', 'sodium_unit', 'energy_value', 'energy_unit', 'off:food_groups', 'off:food_groups_tags', 'off:nova_groups', 'off:nova_groups_tags', 'off:nutriscore_grade', 'off:nutriscore_score', 'off:ecoscore_grade', 'off:ecoscore_score', 'off:ecoscore_data.missing_key_data', 'off:ecoscore_data.agribalyse.code', 'off:ecoscore_data.adjustments.origins_of_ingredients.value', 'off:ecoscore_data.adjustments.packaging.value', 'off:ecosc

In [14]:
data_filtered_columns.loc[0, 'countries_tags']

'en:belgium,en:france,en:french-polynesia,en:germany,en:guadeloupe,en:italy,en:martinique,en:reunion,en:slovenia,en:spain,en:switzerland,en:united-kingdom'

In [15]:
# Accessing the 'countries' column from the first row of the DataFrame
countries_value = data_filtered_columns.loc[0, 'countries']

# Splitting the values if they are strings
if isinstance(countries_value, str):
    countries_list = countries_value.split(',')
    print(countries_list)
else:
    print("Value is not a string.")

['Belgique', ' France', ' Polynésie française', ' Allemagne', ' Guadeloupe', ' Italie', ' Martinique', ' La Réunion', ' Slovénie', ' Espagne', ' Suisse', ' Royaume-Uni']


In [16]:
# Save the current DataFrame to a new CSV file
data_filtered_columns.to_csv('cereal_spain_cleaned_v0.csv', index=False)